# FEniCSx计算框架
---
<div style="float: left; clear: both;" align="left">
<img src="https://fenicsproject.org/assets/img/fenics-logo.png" width="180" alt="fenics-logo.png" align=left hspace="5" vspace="5"/>
<br /><br />
FEniCS项目是一个研究和软件项目，旨在创建用于解决偏微分方程的数学方法和软件。这包括创建直观、高效和灵活的软件。该项目于2003年启动，由来自世界各地一些大学和研究机构的研究人员合作开发。有关FEniCS项目的最新进展和更多信息，请访问<a href="https://fenicsproject.org" title="FEniCS网站">FEniCS网站</a>。
<br /><br />
FEniCS项目的最新版本FEniCSx由几个构建模块组成，即Basix、UFL、FFCx和DOLFINx。DOLFINx是FEniCSx的高性能C++后端，网格、函数空间和函数等结构在这里实现。此外，DOLFINx还包含计算密集型功能，如有限元组装和网格细化算法。它还提供了与线性代数求解器和数据结构的接口，如PETSc。UFL是一种高级形式语言，用于描述具有高级数学语法的变分公式。FFCx是FEniCSx的形式编译器，给定用UFL编写的变量公式，它可以生成高效的C代码。Basix是FEniCSx的有限元后端，负责生成有限元基函数。
<br /><br />
</div>

---
### 参考资料

[The FEniCSx tutorial](https://jorgensd.github.io/dolfinx-tutorial/fem.html)

## 安装FEniCSx
---

FEniCSx库对外部底层库的依赖非常复杂，因此FEniCSx通常作为独立的环境进行打包和安装。相对于复杂的源码编译，我们将使用官方提供的[Dolfinx Docker容器](https://hub.docker.com/r/dolfinx/dolfinx)。

Docker 是一个开源的应用容器引擎，让开发者可以打包他们的应用以及依赖包到一个可移植的镜像中，然后发布到任何流行的 Linux或Windows操作系统的机器上，也可以实现虚拟化。容器是完全使用沙箱机制，相互之间不会有任何接口。

**官方镜像**
```shell
docker run dolfinx/dolfinx
```

**包含JupyterLab的定制镜像**
```shell
docker run -d --name dolfinx --init -p 8888:8888 -v "$(pwd)":/root/shared dokken92/dolfinx_custom:labv0.4.0
```

Docker的具体使用请参看[官方教程](https://docs.docker.com/get-started/)。

## [Poisson equation](https://docs.fenicsproject.org/dolfinx/main/python/demos/demo_poisson.html)
---

For a domain $\Omega \subset \mathbb{R}^n$ with boundary $\partial \Omega$ = $\Gamma_{D} \cup \Gamma_{N}$, the Poisson equation with particular boundary conditions reads:

$$\begin{split} \begin{align} - \nabla^{2} u &= f \quad {\rm in} \ \Omega, \\ u &= 0 \quad {\rm on} \ \Gamma_{D}, \\ \nabla u \cdot n &= g \quad {\rm on} \ \Gamma_{N}. \\ \end{align} \end{split}$$

where $f$ and $g$ are input data and $n$ denotes the outward directed boundary normal. The variational problem reads: find $u \in V$ such that

$$a(u, v) = L(v) \quad \forall  v \in V$$

where $V$ is a suitable function space and

$$\begin{align}
a(u, v) &:= \int_{\Omega} \nabla u \cdot \nabla v \, {\rm d} x, \\
L(v)    &:= \int_{\Omega} f v \, {\rm d} x + \int_{\Gamma_{N}} g v \, {\rm d} s.
\end{align}$$

The expression $a(u, v)$ is the bilinear form and $L(v)$is the linear form. It is assumed that all functions in $V$ satisfy the Dirichlet boundary conditions ($u = 0 \ {\rm on} \ \Gamma_{D}$).

In this demo we consider:
$$\Omega = [0,2] \times [0,1]$$
$$\Gamma_{D} = \{(0, y) \cup (1, y) \subset \partial \Omega\}$$
$$\Gamma_{N} = \{(x, 0) \cup (x, 1) \subset \partial \Omega\}$$
$$g = \sin(5x)$$
$$f = 10\exp(-((x - 0.5)^2 + (y - 0.5)^2) / 0.02)$$

In [1]:
import numpy as np

import ufl
from dolfinx import fem, io, mesh, plot
from ufl import ds, dx, grad, inner

from mpi4py import MPI
from petsc4py.PETSc import ScalarType

In [2]:
'''
# We begin by using {py:func}`create_rectangle
# <dolfinx.mesh.create_rectangle>` to create a rectangular
# {py:class}`Mesh <dolfinx.mesh.Mesh>` of the domain, and creating a
# finite element {py:class}`FunctionSpace <dolfinx.fem.FunctionSpace>`
# $V$ on the mesh.
'''
msh = mesh.create_rectangle(comm=MPI.COMM_WORLD,
                            points=((0.0, 0.0), (2.0, 1.0)), n=(32, 16),
                            cell_type=mesh.CellType.triangle,)
V = fem.FunctionSpace(msh, ("Lagrange", 1))

In [3]:
'''
# The second argument to {py:class}`FunctionSpace
# <dolfinx.fem.FunctionSpace>` is a tuple consisting of `(family,
# degree)`, where `family` is the finite element family, and `degree`
# specifies the polynomial degree. in this case `V` consists of
# first-order, continuous Lagrange finite element functions.
#
# Next, we locate the mesh facets that lie on the boundary $\Gamma_D$.
# We do this using using {py:func}`locate_entities_boundary
# <dolfinx.mesh.locate_entities_boundary>` and providing  a marker
# function that returns `True` for points `x` on the boundary and
# `False` otherwise.
'''
facets = mesh.locate_entities_boundary(msh, dim=1,
                                       marker=lambda x: np.logical_or(np.isclose(x[0], 0.0),
                                                                      np.isclose(x[0], 2.0)))

In [4]:
'''
# We now find the degrees-of-freedom that are associated with the
# boundary facets using {py:func}`locate_dofs_topological
# <dolfinx.fem.locate_dofs_topological>`
'''
dofs = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facets)

In [5]:
'''
# and use {py:func}`dirichletbc <dolfinx.fem.dirichletbc>` to create a
# {py:class}`DirichletBCMetaClass <dolfinx.fem.DirichletBCMetaClass>`
# class that represents the boundary condition
'''
bc = fem.dirichletbc(value=ScalarType(0), dofs=dofs, V=V)

In [6]:
'''
# Next, we express the variational problem using UFL.
'''
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
x = ufl.SpatialCoordinate(msh)
f = 10 * ufl.exp(-((x[0] - 0.5) ** 2 + (x[1] - 0.5) ** 2) / 0.02)
g = ufl.sin(5 * x[0])
a = inner(grad(u), grad(v)) * dx
L = inner(f, v) * dx + inner(g, v) * ds

In [7]:
'''
# We create a {py:class}`LinearProblem <dolfinx.fem.LinearProblem>`
# object that brings together the variational problem, the Dirichlet
# boundary condition, and which specifies the linear solver. In this
# case we use a direct (LU) solver. The {py:func}`solve
# <dolfinx.fem.LinearProblem.solve>` will compute a solution.
'''
problem = fem.petsc.LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [8]:
'''
# The solution can be written to a  {py:class}`XDMFFile
# <dolfinx.io.XDMFFile>` file visualization with ParaView ot VisIt
'''
with io.XDMFFile(msh.comm, "out_poisson/poisson.xdmf", "w") as file:
    file.write_mesh(msh)
    file.write_function(uh)

In [9]:
'''
# and displayed using [pyvista](https://docs.pyvista.org/).
'''
try:
    import pyvista
    pyvista.set_jupyter_backend('ipygany')
    cells, types, x = plot.create_vtk_mesh(V)
    grid = pyvista.UnstructuredGrid(cells, types, x)
    grid.point_data["u"] = uh.x.array.real
    grid.set_active_scalars("u")
    plotter = pyvista.Plotter()
    plotter.add_mesh(grid, show_edges=True)
    warped = grid.warp_by_scalar()
    plotter.add_mesh(warped)
    plotter.show()
except ModuleNotFoundError:
    print("'pyvista' is required to visualise the solution")
    print("Install 'pyvista' with pip: 'python3 -m pip install pyvista'")

AppLayout(children=(VBox(children=(HTML(value='<h3>u</h3>'), Dropdown(description='Colormap:', options={'BrBG'…